In [ ]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 4.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=39793c1771f9519d55391c918e9137080064e0091a590b2e338646b3a94f021b
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting unin

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
     QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from functools import partial
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit
from sklearn.base import clone
from ISLP.models import sklearn_sm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample

In [ ]:
df = load_data('Weekly')
df.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [ ]:
X = df[['Lag1', 'Lag2']]
y = (df['Direction'] == 'Up').astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

kf = KFold(n_splits = X_train.shape[0], shuffle = True, random_state = 2025)

predictions = []
actual_values = []

for train_id, val_id in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_id], X_train.iloc[val_id]
    y_train_fold, y_val_fold = y_train.iloc[train_id], y_train.iloc[val_id]

    model = sm.Logit(y_train_fold, X_train_fold).fit(disp = False)
    model_preds_prob = model.predict(X_val_fold)
    model_preds = (model_preds_prob > 0.5).astype(int)

    predictions.append(model_preds.iloc[0])
    actual_values.append(y_val_fold.iloc[0])

train_accuracy = accuracy_score(actual_values, predictions)
print(f'LOOCV Accuracy on Training Set: {train_accuracy:.4f}')


final_model = sm.Logit(y_train, X_train).fit(disp=False)
y_test_pred_prob = final_model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Set Accuracy: {test_accuracy:.4f}')


LOOCV Accuracy on Training Set: 0.5316
Test Set Accuracy: 0.5092


In [ ]:
X = df[['Lag1', 'Lag2']]
y = (df['Direction'] == 'Up').astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

question_1 = X_train.iloc[0]
answer_1 = y_train.iloc[0]

X_train = X_train.iloc[1:]
y_train = y_train.iloc[1:]

kf = KFold(n_splits = X_train.shape[0], shuffle = True, random_state = 2025)

predictions = []
actual_values = []

for train_id, val_id in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_id], X_train.iloc[val_id]
    y_train_fold, y_val_fold = y_train.iloc[train_id], y_train.iloc[val_id]

    model = sm.Logit(y_train_fold, X_train_fold).fit(disp = False)
    model_preds_prob = model.predict(X_val_fold)
    model_preds = (model_preds_prob > 0.5).astype(int)

    predictions.append(model_preds.iloc[0])
    actual_values.append(y_val_fold.iloc[0])

print(f'Model without the first record')
train_accuracy = accuracy_score(actual_values, predictions)
print(f'LOOCV Accuracy on Training Set: {train_accuracy:.4f}')


final_model = sm.Logit(y_train, X_train).fit(disp=False)
y_test_pred_prob = final_model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Set Accuracy: {test_accuracy:.4f}')








Model without the first record
LOOCV Accuracy on Training Set: 0.5333
Test Set Accuracy: 0.5092


In [ ]:
final_model = sm.Logit(y_train, (X_train)).fit(disp=False)
pred_prob_answer1 = final_model.predict(question_1)
pred_answer = (pred_prob_answer1 > 0.5).astype(int)

# Verificamos si la predicción fue correcta
if pred_answer.iloc[0] == answer_1:
    print(f'The answer is correct and the prediction was {pred_answer.iloc[0]}')
else:
    print(f'The answer is incorrect and the prediction was {pred_answer.iloc[0]} --- real answer: {answer_1}')

The answer is incorrect and the prediction was 1 --- real answer: 0


#D

In [ ]:
X = df[['Lag1', 'Lag2']].values
y = (df['Direction'] == 'Up').astype(int).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2025)


#LOOCV
n_train = len(X_train)
error_vector = np.zeros(n_train)


for i in range(n_train):

   #D.a
    X_train_cv = np.delete(X_train, i, axis=0)
    y_train_cv = np.delete(y_train, i)
    X_val = X_train[i].reshape(1, -1)
    y_val = y_train[i]

    #D.b
    model = LogisticRegression(random_state=2025)
    model.fit(X_train_cv, y_train_cv)
    prob_up = model.predict_proba(X_val)[0][1]

    #D,c
    prediction = prob_up > 0.5

    #D.d
    error_vector[i] = int(prediction != y_val)


cv_error_rate = np.mean(error_vector)

final_model = LogisticRegression(random_state=42)
final_model.fit(X_train, y_train)

# Make predictions on test set
test_predictions = final_model.predict(X_test)
test_error_rate = np.mean(test_predictions != y_test)

print(f"Cross-validation error rate on training data: {cv_error_rate:.3f}")
print(f"Test set error rate: {test_error_rate:.3f}")






Cross-validation error rate on training data: 0.448
Test set error rate: 0.440


# Conclusion:

This CV methods are some techniques used in order to evaluate the performace of the model in a more robust way but they aint made to work for a final fit (Dont expect using cv to start predicting).

CV is most used as hyperparameter tunning and vars selection.

Diferent cv approach can help you depending the number of rows in your data and type of problem your working with (In medicine you would like that your results are the most robust as possible).
